In [36]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver
from llm_config import get_llm, get_embeddings

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
graph = Neo4jGraph()

C:\Users\TRONG TINH\AppData\Local\Temp\ipykernel_33212\267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [3]:
loader = TextLoader(file_path="dummytext_vi.txt", encoding="utf-8")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

dummytext.txt → đọc toàn bộ nội dung → tách nhỏ ra nhiều document (~250 ký tự mỗi document, có overlap 24 ký tự)



In [4]:

llm = get_llm()

llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)


documents (text chunks) → Gemini (LLM) → entity + relation extraction → graph_documents


In [5]:
graph_documents[0]

GraphDocument(nodes=[], relationships=[], source=Document(metadata={'source': 'dummytext_vi.txt'}, page_content='1. Câu Chuyện Gia Đình Amico: Di Sản của Tình Yêu và Truyền Thống'))

In [6]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

graph_documents → add_graph_documents → Neo4j:

    - Tạo các Node (entities)

    - Tạo các Relationship (edges)

    - Thêm label "Entity" nếu bật baseEntityLabel
    
    - Thêm metadata source nếu bật include_source


In [7]:
embeddings = get_embeddings()

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)
vector_retriever = vector_index.as_retriever()

[Neo4j Graph DB]

       ↑

(LLM sinh entity + relation → lưu vào Graph)

       ↓

[Neo4j Vector Index]

       ↑

(embedding của text trong Document node → lưu vào embedding field)

       ↓
       
[User query → vector_retriever → top-k relevant documents → LLM answer]


In [8]:
driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

In [9]:

class Entities(BaseModel):
    """Identifying information about entities."""

    names: list[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)


entity_chain = llm.with_structured_output(Entities)

In [10]:
entity_chain.invoke("Nonna Lucia và Giovanni Caruso là ai?")

Entities(names=['Nonna Lucia', 'Giovanni Caruso'])

In [11]:
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    if not words:
        return ""
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    print(f"Generated Query: {full_text_query}")
    return full_text_query.strip()


# Fulltext index query
def graph_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke(question)
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [12]:
print(graph_retriever("Nonna Lucia là ai?"))

Nonna Lucia - TEACH -> Gian Thiêng Liêng
Nonna Lucia - GUIDE -> Gia Đình
Nonna Lucia - IS -> Người Hướng Dẫn
Nonna Lucia - IS -> Người Mẹ
Nonna Lucia - TEACH -> Sicilian Cooking
Nonna Lucia - INFLUENCE -> Amico
Nonna Lucia - GRANDPARENT -> Amico
Nonna Lucia - MENTOR -> Nonna Lucia
Nonna Lucia - MATRIARCH -> Nonna Lucia
Amico - INFLUENCE -> Nonna Lucia
Nonna Lucia - MENTOR -> Nonna Lucia
Nonna Lucia - MATRIARCH -> Nonna Lucia
Lucia - FOCUS -> Cooking Methods
Lucia - SUPPORTS -> Bella Vita
Lucia - RELATED_TO -> Baking
Lucia - FOCUSES_ON -> Sustainable Cooking Methods
Lucia - RELATED_TO -> Seafood Processing
Lucia - COMMITMENT -> Sustainability
Lucia - SUPPORT -> Artist
Lucia - TEACH -> Đầu Bếp Trẻ
Lucia - LOCATION -> Los Angeles
Lucia - SUPPORTS -> Artists
Lucia - OWNS -> Bella Vita
Lucia - GRANDMOTHER -> Amico
Lucia - SISTER -> Antonio
Lucia - MOTHER -> Caruso Family
Lucia - FOCUSES_ON -> Sustainable Cooking Practices
Lucia - LOCATED_IN -> Los Angeles
Lucia - TEACHES -> Local Ingredient

In [13]:
def full_retriever(question: str):
    graph_data = graph_retriever(question)
    vector_data = [el.page_content for el in vector_retriever.invoke(question)]
    final_data = f"""Graph data:
{graph_data}
vector data:
{"#Document ". join(vector_data)}
    """
    return final_data

In [14]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
        {
            "context": full_retriever,
            "question": RunnablePassthrough(),
        }
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
chain.invoke(input="Nonna Lucia là ai? Bà ấy có dạy ai về nhà hàng hay nấu ăn không?")

'Nonna Lucia là Người Mẹ và Người Hướng Dẫn. Bà dạy Gian Thiêng Liêng và Sicilian Cooking.'

In [16]:
# Thử với một số câu hỏi đơn giản
test_questions = [
    "Câu chuyện chính là gì?",
    "Nhân vật chính là ai?",
    "Những sự kiện chính trong câu chuyện là gì?"
]

for question in test_questions:
    print(f"\nQuestion: {question}")
    answer = chain.invoke(question)
    print(f"Answer: {answer}")


Question: Câu chuyện chính là gì?
Answer: Câu chuyện chính là về Antonio, một người kể chuyện có sức hút, người thừa hưởng niềm đam mê từ cha mẹ và nổi tiếng với việc thu hút gia đình và làng xóm bằng những câu chuyện.

Question: Nhân vật chính là ai?
Answer: Không có đủ thông tin để xác định nhân vật chính. Các đoạn văn bản chỉ đề cập đến những người gìn giữ di sản ẩm thực, một gia đình có đóng góp vào di sản ẩm thực, và một nông dân cùng vợ.

Question: Những sự kiện chính trong câu chuyện là gì?
Answer: Các sự kiện chính là các buổi biểu diễn trực tiếp, trình diễn nấu ăn và chợ thủ công.


In [19]:
# Kiểm tra cấu trúc graph đã được tạo
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 10
"""
result = graph.query(query)
print("Graph Structure:")
print(result)

Graph Structure:
[{'n': {'id': '3d29be2f7e79f8b2f67f8e4f17cd95a7', 'source': 'dummytext.txt', 'text': 'Giovanni Caruso and Maria: The Founding Generation', 'embedding': [0.0361039824783802, -0.005127072799950838, -0.051021188497543335, 0.033234383910894394, 0.08429624140262604, 0.0423525869846344, 0.019594620913267136, -0.021883733570575714, -0.031491268426179886, 0.045937199145555496, 0.012146524153649807, 0.024090996012091637, -0.04528249055147171, 0.019265003502368927, 0.05319961532950401, 0.016839126124978065, 0.019627880305051804, -0.007014109753072262, -0.007494767662137747, -0.034221649169921875, 0.003947795368731022, 0.028194023296236992, -0.004558344837278128, -0.03069627657532692, -0.01882964000105858, 0.019545970484614372, 0.019715188071131706, -0.05440877750515938, -0.03644811362028122, -0.028351623564958572, -0.06878183782100677, -0.005040577147156, -0.060017384588718414, 0.046373527497053146, 0.04312969371676445, -0.03661968559026718, 0.013651693239808083, -0.006772298831

In [18]:
# Test vector search với một câu hỏi
test_query = "Câu chuyện nói về điều gì??"
results = vector_retriever.get_relevant_documents(test_query)
print("\nRelevant Documents:")
for doc in results:
    print(f"\nContent: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    

C:\Users\TRONG TINH\AppData\Local\Temp\ipykernel_33212\2164379090.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = vector_retriever.get_relevant_documents(test_query)



Relevant Documents:

Content: 
text: món ăn quốc tế khác. Những lễ hội này có các buổi biểu diễn trực tiếp, trình diễn nấu ăn và chợ thủ công, tạo nên một bầu không khí sôi động và bao trùm. Các sự kiện không chỉ là một lễ kỷ niệm về ẩm thực mà còn là một nền tảng cho giao lưu văn hóa,
Metadata: {'source': 'dummytext_vi.txt'}

Content: 
text: 1. Câu Chuyện Gia Đình Amico: Di Sản của Tình Yêu và Truyền Thống
Metadata: {'source': 'dummytext_vi.txt'}

Content: 
text: hưởng của Nonna Lucia vượt ra ngoài căn bếp; bà là trụ cột của sức mạnh và trí tuệ, hướng dẫn gia đình bằng tình yêu và bàn tay vững vàng. Những bài học của bà vượt xa các công thức nấu ăn, truyền dạy những bài học về cuộc sống, tình yêu và tầm quan
Metadata: {'source': 'dummytext_vi.txt'}

Content: 
text: ấn không thể xóa nhòa trong thế giới ẩm thực. Mỗi thành viên trong gia đình, với nền tảng và tài năng độc đáo của mình, đã đóng góp vào di sản của gia đình, tạo nên một đế chế ẩm thực trải dài qua nhiều lục địa. Cam kết củ